# 道路追踪-现场演示

In [1]:
#---------------------------识别两次动物，第二次停止------------------------------------------------------------
#-----2020-11-04--------这个代码拍下哦一个不错的视频，先识别长颈鹿，看到就停下10秒钟；再看到狗就彻底停下-----------
#-----2020-11-06--------斑马、马的效果也很好---------大象效果也很好-------------------------------------------------

import torchvision
import torch
import time
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import bgr8_to_jpeg
from jetbot import Robot
from jetbot import Camera
camera = Camera.instance(width=224, height=224,fps=10)
device = torch.device('cuda')

from jetbot import ObjectDetector
model_object = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [2]:
robot = Robot()
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

model.load_state_dict(torch.load('huangsexian_304zhang_50epoch.pth'))#黄色中间线模型
#model.load_state_dict(torch.load('huangsexian_200zhang.pth'))#黄色中间线模型

model = model.to(device)
model = model.eval().half()
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

In [3]:
#这里是我定义的---20201008
def find_object(objectId):
    #objectId = 16是鸟、17是猫、18是狗、19是马、20是羊、21是牛、22是大象、23是熊、24是斑马、25是长颈鹿
    image = camera.value
    image = cv2.resize(image,(300,300))
    detections = model_object(image)
    matching_detections = [d for d in detections[0] if d['label'] == int(objectId)]
    
    if(len(matching_detections) != 0):
        return True
    else:
        return False

In [4]:
def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

现在，我们将定义滑块来控制JetBot
1. 速度控制（speed_gain_slider）：开始增加``speed_gain_slider``来启动JetBot
2. 转向增益控制（steering_gain_sloder）：如果您看到JetBot摇摆不定，则需要减小``steering_gain_slider``直到平稳为止
3. 转向偏差控制（steering_bias_slider）：如果看到JetBot偏向轨道的最右侧或最左侧，则应控制此滑块，直到JetBot开始沿中心线或轨道行进。这说明了电机偏差以及相机偏移

In [5]:
#这个是把精度和范围变小的，适用于微调---黄色中间线
#speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.29, description='speed gain')
#steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.2, step=0.001, value=0.04, description='steering gain')
#steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.2, step=0.001, value=0.05, description='steering kd')
#steering_bias_slider = ipywidgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=0.03, description='steering bias')

#这个是把精度和范围变小的，适用于微调
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.47, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.2, step=0.001, value=0.09, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.2, step=0.001, value=0.03, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=0.01, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.47, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.09, description='steering gain', max=0.2, step=0.001)

FloatSlider(value=0.03, description='steering kd', max=0.2, step=0.001)

FloatSlider(value=0.01, description='steering bias', max=0.1, min=-0.1, step=0.001)

In [6]:
x_slider = ipywidgets.FloatSlider(min=-100, max=100, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

In [7]:
angle = 0.0
angle_last = 0.0
stopped = 0
road_or_animal = 0

def execute(image):
    global angle, angle_last
    global stopped
    global road_or_animal
    road_or_animal += 1
   
    if stopped < 2:#寻路，发现目标就停下
        if road_or_animal%4 != 100:#1帧寻路1帧识别，交叉运行
            #image = cv2.resize(image,(224,224))#因为识别动物的要求高，所以获取到300*300的要缩小一下
            xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0
            x_slider.value = x
            y_slider.value = y
            #print('x: '+str(x) + ' y: '+ str(y))
            speed_slider.value = speed_gain_slider.value
            angle = np.arctan2(x, y)
            pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
            angle_last = angle
            #steering_slider.value = pid + steering_bias_slider.value
            #robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 0.5), 0.0)
            #robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 0.5), 0.0)
            steering_slider.value = pid + steering_bias_slider.value
            my_value0 = 0.5
            my_value = 0.5
            robot.left_motor.value = max(min(speed_slider.value*my_value0 + steering_slider.value*my_value, 1), 0.0)
            robot.right_motor.value = max(min(speed_slider.value*my_value0 - steering_slider.value*my_value, 1), 0.0)
                
        else:
            if stopped == 0:#第一次停止
                if find_object(objectId = 18) == True:
                    print('小车停一下')#图像是300x300的可以识别大象，224x224的识别错误
                    #16是鸟、17是猫、18是狗、19是马、20是羊、21是牛、22是大象、23是熊、24是斑马、25是长颈鹿
                    stopped = 2#1
                    robot.left_motor.value = 0
                    robot.right_motor.value = 0
                    time.sleep(10)
            else:
                if stopped == 1:
                    if find_object(objectId = 18) == True:#第二次的动物，25是长颈鹿
                        print('彻底停止了')
                        stopped = 2#速度设置为0后跳出执行
                        robot.left_motor.value = 0
                        robot.right_motor.value = 0

In [8]:
count = 0
def switch(change):
    global count
    count += 1
    if count <= 6:
        robot.left_motor.value = 0.29
        robot.right_motor.value = 0.29
        return 0
    image = change['new']
    execute(image)

In [9]:
camera.observe(switch, names='value')

In [10]:
#--------后面是停掉相机--------

In [11]:
camera.unobserve(switch, names='value')

In [12]:
camera.stop()

In [13]:
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

In [14]:
camera.stop()